In [ ]:
import matplotlib.pyplot as plt
import re
import os
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import gensim

## Load Data

In [ ]:
data = pd.read_csv("final test.csv")

In [ ]:
# data = data.loc[data['Job Title final']=='Data Scientist']
# data = data.loc[data['Job Title final']=='Data Engineer']
# data = data.loc[data['Job Title final']=='Data Analyst']
data = data.loc[data['Job Title final']=='Business Analyst']

In [ ]:
data

,Unnamed: 0,Unnamed: 0.1,Job Title,Job Title final,Salary Estimate,Salary_Estimate_low,Salary_Estimate_high,Salary_Estimate_average,Job Description,Rating,...,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,low,high,mid,binary_avg
11576,0,12034,Business Analyst - Clinical & Logistics Platform,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,Company Overview At Memorial Sloan Kettering M...,3.9,...,Nonprofit Organization,Health Care Services & Hospitals,Health Care,2 to 5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1,3,3,3,2
11577,1,12035,Business Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,We are seeking for an energetic and collaborat...,3.8,...,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,TRUE,3,3,3,2
11578,2,12036,Data Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,For more than a decade Asembia ha been working...,3.6,...,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,5 to 10 million (USD),-1,-1,3,3,3,2
11579,3,12037,"Information Security Analyst, Incident Response",Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,Job Description Summary The Information Securi...,3.6,...,Company - Public,Health Care Products Manufacturing,Manufacturing,10+ billion (USD),"Abbott, Siemens, Baxter",-1,3,3,3,2
11580,4,12038,Analyst - FP&A Global Revenue,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,Magnite is the world s largest independent sel...,3.4,...,Company - Public,Internet,Information Technology,100 to 500 million (USD),"PubMatic, AppNexus, Index Exchange",TRUE,3,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15098,3687,15721,Pega Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,yr relevant experience in Experience with Pega...,3.1,...,Company - Private,Consulting,Business Services,5 to 10 million (USD),-1,-1,3,3,3,2
15099,3688,15722,Jr.Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,Job Description New business are making their ...,5.0,...,Company - Private,IT Services,Information Technology,50 to 100 million (USD),-1,-1,3,3,3,2
15100,3689,15723,Business Process Analyst/Remote 4 mth+ contract,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,Title Business Process Analyst Duration Months...,-1.0,...,-1,-1,-1,-1,-1,-1,3,3,3,2
15101,3690,15724,"AVP, Business System",Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,We are looking for a AVP Business System Analy...,3.5,...,Company - Public,Investment Banking & Asset Management,Finance,2 to 5 billion (USD),"Fidelity Investments, UBS, Morgan Stanley",-1,3,3,3,2


In [ ]:
data.info() 
#no further preprocessing for null values because of no null values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3527 entries, 11576 to 15102
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               3527 non-null   int64  
 1   Unnamed: 0.1             3527 non-null   int64  
 2   Job Title                3527 non-null   object 
 3   Job Title final          3527 non-null   object 
 4   Salary Estimate          3527 non-null   object 
 5   Salary_Estimate_low      3527 non-null   int64  
 6   Salary_Estimate_high     3527 non-null   int64  
 7   Salary_Estimate_average  3527 non-null   float64
 8   Job Description          3527 non-null   object 
 9   Rating                   3527 non-null   float64
 10  Company Name             3527 non-null   object 
 11  Location_city            3527 non-null   object 
 12  Location_state           3527 non-null   object 
 13  Headquarters             3527 non-null   object 
 14  Size               

In [ ]:
data = data.dropna()

In [ ]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3527 entries, 11576 to 15102
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               3527 non-null   int64  
 1   Unnamed: 0.1             3527 non-null   int64  
 2   Job Title                3527 non-null   object 
 3   Job Title final          3527 non-null   object 
 4   Salary Estimate          3527 non-null   object 
 5   Salary_Estimate_low      3527 non-null   int64  
 6   Salary_Estimate_high     3527 non-null   int64  
 7   Salary_Estimate_average  3527 non-null   float64
 8   Job Description          3527 non-null   object 
 9   Rating                   3527 non-null   float64
 10  Company Name             3527 non-null   object 
 11  Location_city            3527 non-null   object 
 12  Location_state           3527 non-null   object 
 13  Headquarters             3527 non-null   object 
 14  Size               

## Data Preprocessing

In [ ]:
# remove non alphabets
remove_non_alphabets = lambda x: re.sub(r'[^a-zA-Z]',' ',x)

# tokenn alphabets-only list
tokenize = lambda x: word_tokenize(x)

#Stopwords Removal and only keep text data then change to lowercase
mystopwords = stopwords.words('english')
spwords = lambda tokens:[w.lower() for w in tokens if w.isalpha() if w.lower()not in mystopwords]

# assign ps to a lambda function to run on each line of value
ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]



lemmatizer = WordNetLemmatizer()
lemmtizer = lambda x: [ lemmatizer.lemmatize(word) for word in x ]

In [ ]:
# To show full text of each tweet
# pd.set_option('display.max_colwidth', None)

In [ ]:
data['Job Description'] = data['Job Description'].apply(remove_non_alphabets)
data

,Unnamed: 0,Unnamed: 0.1,Job Title,Job Title final,Salary Estimate,Salary_Estimate_low,Salary_Estimate_high,Salary_Estimate_average,Job Description,Rating,...,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,low,high,mid,binary_avg
11576,0,12034,Business Analyst - Clinical & Logistics Platform,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,Company Overview At Memorial Sloan Kettering M...,3.9,...,Nonprofit Organization,Health Care Services & Hospitals,Health Care,2 to 5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1,3,3,3,2
11577,1,12035,Business Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,We are seeking for an energetic and collaborat...,3.8,...,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,TRUE,3,3,3,2
11578,2,12036,Data Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,For more than a decade Asembia ha been working...,3.6,...,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,5 to 10 million (USD),-1,-1,3,3,3,2
11579,3,12037,"Information Security Analyst, Incident Response",Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,Job Description Summary The Information Securi...,3.6,...,Company - Public,Health Care Products Manufacturing,Manufacturing,10+ billion (USD),"Abbott, Siemens, Baxter",-1,3,3,3,2
11580,4,12038,Analyst - FP&A Global Revenue,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,Magnite is the world s largest independent sel...,3.4,...,Company - Public,Internet,Information Technology,100 to 500 million (USD),"PubMatic, AppNexus, Index Exchange",TRUE,3,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15098,3687,15721,Pega Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,yr relevant experience in Experience with Pega...,3.1,...,Company - Private,Consulting,Business Services,5 to 10 million (USD),-1,-1,3,3,3,2
15099,3688,15722,Jr.Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,Job Description New business are making their ...,5.0,...,Company - Private,IT Services,Information Technology,50 to 100 million (USD),-1,-1,3,3,3,2
15100,3689,15723,Business Process Analyst/Remote 4 mth+ contract,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,Title Business Process Analyst Duration Months...,-1.0,...,-1,-1,-1,-1,-1,-1,3,3,3,2
15101,3690,15724,"AVP, Business System",Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,We are looking for a AVP Business System Analy...,3.5,...,Company - Public,Investment Banking & Asset Management,Finance,2 to 5 billion (USD),"Fidelity Investments, UBS, Morgan Stanley",-1,3,3,3,2


In [ ]:
data['Job Description'] = data['Job Description'].apply(tokenize)
data

,Unnamed: 0,Unnamed: 0.1,Job Title,Job Title final,Salary Estimate,Salary_Estimate_low,Salary_Estimate_high,Salary_Estimate_average,Job Description,Rating,...,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,low,high,mid,binary_avg
11576,0,12034,Business Analyst - Clinical & Logistics Platform,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[Company, Overview, At, Memorial, Sloan, Kette...",3.9,...,Nonprofit Organization,Health Care Services & Hospitals,Health Care,2 to 5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1,3,3,3,2
11577,1,12035,Business Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[We, are, seeking, for, an, energetic, and, co...",3.8,...,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,TRUE,3,3,3,2
11578,2,12036,Data Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[For, more, than, a, decade, Asembia, ha, been...",3.6,...,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,5 to 10 million (USD),-1,-1,3,3,3,2
11579,3,12037,"Information Security Analyst, Incident Response",Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[Job, Description, Summary, The, Information, ...",3.6,...,Company - Public,Health Care Products Manufacturing,Manufacturing,10+ billion (USD),"Abbott, Siemens, Baxter",-1,3,3,3,2
11580,4,12038,Analyst - FP&A Global Revenue,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[Magnite, is, the, world, s, largest, independ...",3.4,...,Company - Public,Internet,Information Technology,100 to 500 million (USD),"PubMatic, AppNexus, Index Exchange",TRUE,3,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15098,3687,15721,Pega Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,"[yr, relevant, experience, in, Experience, wit...",3.1,...,Company - Private,Consulting,Business Services,5 to 10 million (USD),-1,-1,3,3,3,2
15099,3688,15722,Jr.Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,"[Job, Description, New, business, are, making,...",5.0,...,Company - Private,IT Services,Information Technology,50 to 100 million (USD),-1,-1,3,3,3,2
15100,3689,15723,Business Process Analyst/Remote 4 mth+ contract,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,"[Title, Business, Process, Analyst, Duration, ...",-1.0,...,-1,-1,-1,-1,-1,-1,3,3,3,2
15101,3690,15724,"AVP, Business System",Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,"[We, are, looking, for, a, AVP, Business, Syst...",3.5,...,Company - Public,Investment Banking & Asset Management,Finance,2 to 5 billion (USD),"Fidelity Investments, UBS, Morgan Stanley",-1,3,3,3,2


In [ ]:
data['Job Description'] = data['Job Description'].apply(spwords)

In [ ]:
import time
program_start = time.time()
data['Job Description'] = data['Job Description'].apply(stem)
program_end = time.time()
print('Total time taken to scrap is {} seconds.'.format(round((program_end - program_start), 2)))
data

Total time taken to scrap is 14.86 seconds.


,Unnamed: 0,Unnamed: 0.1,Job Title,Job Title final,Salary Estimate,Salary_Estimate_low,Salary_Estimate_high,Salary_Estimate_average,Job Description,Rating,...,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,low,high,mid,binary_avg
11576,0,12034,Business Analyst - Clinical & Logistics Platform,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[compani, overview, memori, sloan, ketter, msk...",3.9,...,Nonprofit Organization,Health Care Services & Hospitals,Health Care,2 to 5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1,3,3,3,2
11577,1,12035,Business Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[seek, energet, collabor, analyst, experi, pas...",3.8,...,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,TRUE,3,3,3,2
11578,2,12036,Data Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[decad, asembia, ha, work, specialti, pharmaci...",3.6,...,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,5 to 10 million (USD),-1,-1,3,3,3,2
11579,3,12037,"Information Security Analyst, Incident Response",Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[job, descript, summari, inform, secur, analys...",3.6,...,Company - Public,Health Care Products Manufacturing,Manufacturing,10+ billion (USD),"Abbott, Siemens, Baxter",-1,3,3,3,2
11580,4,12038,Analyst - FP&A Global Revenue,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,"[magnit, world, largest, independ, sell, side,...",3.4,...,Company - Public,Internet,Information Technology,100 to 500 million (USD),"PubMatic, AppNexus, Index Exchange",TRUE,3,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15098,3687,15721,Pega Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,"[yr, relev, experi, experi, pega, rpa, robot, ...",3.1,...,Company - Private,Consulting,Business Services,5 to 10 million (USD),-1,-1,3,3,3,2
15099,3688,15722,Jr.Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,"[job, descript, new, busi, make, way, market, ...",5.0,...,Company - Private,IT Services,Information Technology,50 to 100 million (USD),-1,-1,3,3,3,2
15100,3689,15723,Business Process Analyst/Remote 4 mth+ contract,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,"[titl, busi, process, analyst, durat, month, r...",-1.0,...,-1,-1,-1,-1,-1,-1,3,3,3,2
15101,3690,15724,"AVP, Business System",Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,"[look, avp, busi, system, analyst, experi, ana...",3.5,...,Company - Public,Investment Banking & Asset Management,Finance,2 to 5 billion (USD),"Fidelity Investments, UBS, Morgan Stanley",-1,3,3,3,2


In [ ]:
data['Job Description'] = data['Job Description'].apply(lambda x: ' '.join(x))
data

,Unnamed: 0,Unnamed: 0.1,Job Title,Job Title final,Salary Estimate,Salary_Estimate_low,Salary_Estimate_high,Salary_Estimate_average,Job Description,Rating,...,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,low,high,mid,binary_avg
11576,0,12034,Business Analyst - Clinical & Logistics Platform,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,compani overview memori sloan ketter msk chang...,3.9,...,Nonprofit Organization,Health Care Services & Hospitals,Health Care,2 to 5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1,3,3,3,2
11577,1,12035,Business Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,seek energet collabor analyst experi passion f...,3.8,...,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,TRUE,3,3,3,2
11578,2,12036,Data Analyst,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,decad asembia ha work specialti pharmaci manuf...,3.6,...,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,5 to 10 million (USD),-1,-1,3,3,3,2
11579,3,12037,"Information Security Analyst, Incident Response",Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,job descript summari inform secur analyst memb...,3.6,...,Company - Public,Health Care Products Manufacturing,Manufacturing,10+ billion (USD),"Abbott, Siemens, Baxter",-1,3,3,3,2
11580,4,12038,Analyst - FP&A Global Revenue,Business Analyst,56K-102K (Glassdoor est.),56,102,79.0,magnit world largest independ sell side platfo...,3.4,...,Company - Public,Internet,Information Technology,100 to 500 million (USD),"PubMatic, AppNexus, Index Exchange",TRUE,3,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15098,3687,15721,Pega Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,yr relev experi experi pega rpa robot platform...,3.1,...,Company - Private,Consulting,Business Services,5 to 10 million (USD),-1,-1,3,3,3,2
15099,3688,15722,Jr.Business Analyst,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,job descript new busi make way market rapidli ...,5.0,...,Company - Private,IT Services,Information Technology,50 to 100 million (USD),-1,-1,3,3,3,2
15100,3689,15723,Business Process Analyst/Remote 4 mth+ contract,Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,titl busi process analyst durat month remot lo...,-1.0,...,-1,-1,-1,-1,-1,-1,3,3,3,2
15101,3690,15724,"AVP, Business System",Business Analyst,58K-103K (Glassdoor est.),58,103,80.5,look avp busi system analyst experi analyz bus...,3.5,...,Company - Public,Investment Banking & Asset Management,Finance,2 to 5 billion (USD),"Fidelity Investments, UBS, Morgan Stanley",-1,3,3,3,2


## Split Train-Test Sets

In [ ]:
# split to 30 percent test data and 70 percent train data
# labels can be seen as y, an dependent variable
train_corpus, test_corpus, train_labels, test_labels = train_test_split(data["Job Description"],
                                                                        data["binary_avg"],
                                                                        test_size=0.2,random_state=42)

In [ ]:
print(len(train_corpus))
train_corpus

2821


12108    year experi design implement support sterl om ...
14893    job descript job summari look high power quant...
14154    northrop grumman employe incred opportun work ...
13226    pharmaci b busi oper manag respons overse dail...
13028    extens prior work experi softwar system analys...
                               ...                        
12706    posit summari busi analyst report conn loan op...
12870    dhl express ha award global top employ immedi ...
12436    senior analyst consolid financ system cstabl m...
15083    altern locat atlanta ga georgia austin tx texa...
14750    kawahara nurseri inc http kniplant com third g...
Name: Job Description, Length: 2821, dtype: object

In [ ]:
print(len(test_corpus))
test_corpus

706


11901    ref id classif project manag compens hourli ro...
14093    fleet optim data analyst solar storag track co...
14442    job descript seek talent driven busi analyst j...
12322    respons work stakehold understand busi object ...
11875    busi analyst tool report lo angelesoper produc...
                               ...                        
14459    edmentum commit make easier educ individu lear...
14308    hi mission provid custom trust foundat digit w...
14590    mileston technolog inc lead manag servic profe...
13486    career start look next gener health care leade...
11771    role work rapidli grow homeown team plymouth r...
Name: Job Description, Length: 706, dtype: object

In [ ]:
train_labels

12108    2
14893    1
14154    2
13226    2
13028    1
        ..
12706    1
12870    2
12436    2
15083    2
14750    2
Name: binary_avg, Length: 2821, dtype: int64

In [ ]:
test_labels

11901    2
14093    2
14442    2
12322    2
11875    2
        ..
14459    2
14308    2
14590    2
13486    1
11771    2
Name: binary_avg, Length: 706, dtype: int64

## Feature Representation

### Binary (Bag of words)

In [ ]:
bow_vectorizer_bi=CountVectorizer(min_df=30, max_df=0.90,max_features=5000,ngram_range=(1,1),binary=True)
# print(bow_vectorizer)
#In short, fit is used to train the feature model, once it's trained you can use that model to apply to the text data
bow_train_features_bi = bow_vectorizer_bi.fit_transform(train_corpus)
#apply model to test data
bow_test_features_bi = bow_vectorizer_bi.transform(test_corpus)

In [ ]:
print(bow_train_features_bi.shape)
print(bow_train_features_bi)

(2821, 1880)
  (0, 1874)	1
  (0, 457)	1
  (0, 824)	1
  (0, 1638)	1
  (0, 1547)	1
  (0, 809)	1
  (0, 304)	1
  (0, 287)	1
  (0, 618)	1
  (0, 575)	1
  (0, 647)	1
  (0, 1446)	1
  (0, 1857)	1
  (0, 292)	1
  (0, 1575)	1
  (0, 1)	1
  (0, 293)	1
  (0, 1663)	1
  (0, 625)	1
  (0, 464)	1
  (0, 714)	1
  (0, 500)	1
  (0, 666)	1
  (0, 341)	1
  (0, 36)	1
  :	:
  (2820, 828)	1
  (2820, 646)	1
  (2820, 1698)	1
  (2820, 711)	1
  (2820, 705)	1
  (2820, 1744)	1
  (2820, 1746)	1
  (2820, 1278)	1
  (2820, 1168)	1
  (2820, 1200)	1
  (2820, 545)	1
  (2820, 847)	1
  (2820, 768)	1
  (2820, 1367)	1
  (2820, 806)	1
  (2820, 477)	1
  (2820, 1660)	1
  (2820, 1411)	1
  (2820, 315)	1
  (2820, 1690)	1
  (2820, 1210)	1
  (2820, 1355)	1
  (2820, 1542)	1
  (2820, 936)	1
  (2820, 1515)	1


In [ ]:
print(bow_test_features_bi.shape)
print(bow_test_features_bi)

(706, 1880)
  (0, 1)	1
  (0, 9)	1
  (0, 13)	1
  (0, 19)	1
  (0, 23)	1
  (0, 39)	1
  (0, 57)	1
  (0, 84)	1
  (0, 86)	1
  (0, 88)	1
  (0, 103)	1
  (0, 104)	1
  (0, 105)	1
  (0, 108)	1
  (0, 115)	1
  (0, 127)	1
  (0, 129)	1
  (0, 131)	1
  (0, 146)	1
  (0, 149)	1
  (0, 162)	1
  (0, 183)	1
  (0, 227)	1
  (0, 247)	1
  (0, 250)	1
  :	:
  (705, 1638)	1
  (705, 1644)	1
  (705, 1656)	1
  (705, 1659)	1
  (705, 1666)	1
  (705, 1697)	1
  (705, 1698)	1
  (705, 1709)	1
  (705, 1732)	1
  (705, 1736)	1
  (705, 1757)	1
  (705, 1759)	1
  (705, 1766)	1
  (705, 1774)	1
  (705, 1778)	1
  (705, 1795)	1
  (705, 1802)	1
  (705, 1813)	1
  (705, 1816)	1
  (705, 1831)	1
  (705, 1840)	1
  (705, 1853)	1
  (705, 1857)	1
  (705, 1869)	1
  (705, 1870)	1


In [ ]:
count_array_bi = bow_train_features_bi.toarray()
df_bi = pd.DataFrame(data=count_array_bi,columns = bow_vectorizer_bi.get_feature_names())
print(df_bi.shape)
df_bi.head(20)

(2821, 1880)


,aa,abil,abl,abreast,absenc,academ,acceler,accentur,accept,access,...,written,www,xml,ye,year,yet,york,youll,yr,zone
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
6,0,1,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
9,0,1,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
print(len(bow_vectorizer_bi.get_feature_names()))

1880


### Frequency (Bag of words) 

In [ ]:
bow_vectorizer=CountVectorizer(min_df=30, max_df=0.90,max_features=5000,ngram_range=(1,1))
# print(bow_vectorizer)
#In short, fit is used to train the feature model, once it's trained you can use that model to apply to the text data
bow_train_features = bow_vectorizer.fit_transform(train_corpus)
#apply model to test data
bow_test_features = bow_vectorizer.transform(test_corpus)

In [ ]:
print(bow_train_features.shape)
print(bow_train_features)

(2821, 1880)
  (0, 1874)	2
  (0, 457)	3
  (0, 824)	1
  (0, 1638)	3
  (0, 1547)	3
  (0, 809)	1
  (0, 304)	3
  (0, 287)	3
  (0, 618)	1
  (0, 575)	1
  (0, 647)	1
  (0, 1446)	1
  (0, 1857)	2
  (0, 292)	1
  (0, 1575)	1
  (0, 1)	5
  (0, 293)	1
  (0, 1663)	1
  (0, 625)	1
  (0, 464)	3
  (0, 714)	1
  (0, 500)	1
  (0, 666)	1
  (0, 341)	2
  (0, 36)	1
  :	:
  (2820, 828)	1
  (2820, 646)	1
  (2820, 1698)	1
  (2820, 711)	2
  (2820, 705)	1
  (2820, 1744)	1
  (2820, 1746)	1
  (2820, 1278)	1
  (2820, 1168)	2
  (2820, 1200)	1
  (2820, 545)	1
  (2820, 847)	1
  (2820, 768)	2
  (2820, 1367)	1
  (2820, 806)	1
  (2820, 477)	1
  (2820, 1660)	1
  (2820, 1411)	1
  (2820, 315)	1
  (2820, 1690)	1
  (2820, 1210)	1
  (2820, 1355)	1
  (2820, 1542)	1
  (2820, 936)	1
  (2820, 1515)	1


In [ ]:
print(bow_test_features.shape)
print(bow_test_features)

(706, 1880)
  (0, 1)	2
  (0, 9)	1
  (0, 13)	1
  (0, 19)	1
  (0, 23)	2
  (0, 39)	1
  (0, 57)	1
  (0, 84)	1
  (0, 86)	1
  (0, 88)	1
  (0, 103)	1
  (0, 104)	5
  (0, 105)	3
  (0, 108)	1
  (0, 115)	1
  (0, 127)	1
  (0, 129)	1
  (0, 131)	1
  (0, 146)	1
  (0, 149)	2
  (0, 162)	1
  (0, 183)	1
  (0, 227)	1
  (0, 247)	1
  (0, 250)	1
  :	:
  (705, 1638)	3
  (705, 1644)	1
  (705, 1656)	1
  (705, 1659)	1
  (705, 1666)	1
  (705, 1697)	1
  (705, 1698)	1
  (705, 1709)	3
  (705, 1732)	1
  (705, 1736)	1
  (705, 1757)	1
  (705, 1759)	2
  (705, 1766)	1
  (705, 1774)	1
  (705, 1778)	1
  (705, 1795)	1
  (705, 1802)	1
  (705, 1813)	1
  (705, 1816)	1
  (705, 1831)	1
  (705, 1840)	1
  (705, 1853)	1
  (705, 1857)	4
  (705, 1869)	1
  (705, 1870)	1


In [ ]:
count_array = bow_train_features.toarray()
df = pd.DataFrame(data=count_array,columns = bow_vectorizer.get_feature_names())
print(df.shape)
df.head(20)

(2821, 1880)


,aa,abil,abl,abreast,absenc,academ,acceler,accentur,accept,access,...,written,www,xml,ye,year,yet,york,youll,yr,zone
0,0,5,0,0,0,0,0,0,0,0,...,1,0,0,0,2,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,3,1,0,0,0,0,0,0,0,...,0,1,0,0,2,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
6,0,8,0,0,0,0,0,0,0,1,...,2,0,0,0,1,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
8,0,7,1,0,0,1,0,0,0,0,...,0,0,0,0,3,0,2,0,0,0
9,0,2,1,0,0,0,0,0,1,0,...,0,0,0,0,2,0,0,0,0,0


In [ ]:
print(len(bow_vectorizer.get_feature_names()))

1880


### Tf-Idf

In [ ]:
# build tfidf features' vectorizer and get features\
#Min_df stands for minimum document frequency,the less the number, the more specific the term
#‘l2’: Sum of squares of vector elements is 1

tfidf_vectorizer=TfidfVectorizer(min_df=30, 
                                 max_df=0.90,
                                 max_features=5000,
                                 norm='l2',
                                 smooth_idf=True,
#                                  th_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
tfidf_train_features = tfidf_vectorizer.fit_transform(train_corpus)  
tfidf_test_features = tfidf_vectorizer.transform(test_corpus)  

In [ ]:
count_array1 = tfidf_test_features.toarray()
df = pd.DataFrame(data=count_array1,columns = tfidf_vectorizer.get_feature_names())
print(df.shape)
df.head(20)

#Different from count vectorizer. The scores are normalized to values between 0 and 1 
#so the vectors can  be used directly with machine learning algorithms.

(706, 1880)


,aa,abil,abl,abreast,absenc,academ,acceler,accentur,accept,access,...,written,www,xml,ye,year,yet,york,youll,yr,zone
0,0.0,0.054186,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.051737,...,0.000000,0.056438,0.000000,0.0,0.093301,0.0,0.0,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.020765,0.000000,0.000000,0.0,0.013449,0.0,0.0,0.000000,0.000000,0.000000
2,0.0,0.064763,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.055757,0.0,0.0,0.000000,0.000000,0.000000
3,0.0,0.173391,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.037319,0.0,0.0,0.000000,0.000000,0.000000
4,0.0,0.029269,0.092220,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.050398,0.0,0.0,0.000000,0.000000,0.106087
5,0.0,0.021056,0.066343,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.043863,0.069723,0.0,0.090640,0.0,0.0,0.000000,0.000000,0.000000
6,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.031701,...,0.022067,0.000000,0.000000,0.0,0.014292,0.0,0.0,0.000000,0.000000,0.000000
7,0.0,0.138358,0.016767,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.014147,0.000000,0.000000,0.0,0.018326,0.0,0.0,0.000000,0.000000,0.000000
8,0.0,0.020183,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.038542,...,0.000000,0.042044,0.000000,0.0,0.017376,0.0,0.0,0.219462,0.000000,0.073154
9,0.0,0.042795,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.036844,0.0,0.0,0.000000,0.000000,0.000000


## Define Evaluation Function

In [ ]:
# define a function to evaluate our classification models based on four metrics
# This defined function is also useful in other cases. This is comparing test_y and pred_y. 
# Both contain 1s and 0s.
def get_metrics(true_labels, predicted_labels):
    
    print ('Accuracy:', np.round(                                                    
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        2))
    print ('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels),
                        2))
    print ('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels),
                        2))
    print ('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels),
                        2))

## Define an Easy-to-use Function for Train/Test/Evaluate

In [ ]:
# define a function that trains the model, performs predictions and evaluates the predictions
def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    # build model. fit(x,y)    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    # evaluate model prediction performance   
#     get_metrics(true_labels=test_labels, 
#                 predicted_labels=predictions)
    result = metrics.confusion_matrix(test_labels,predictions)
    print(result)
    cost = result[0][1]*100+result[1][0]*5
    cost1 = cost/(len(test_labels))
    print(cost)
    print(metrics.classification_report(test_labels,predictions))
#     print("\n")
#     print(get_metrics)
    return predictions, metrics.accuracy_score(test_labels,predictions),cost1  

## Import Classifiers

In [ ]:
from sklearn.naive_bayes import MultinomialNB # import naive bayes
from sklearn.tree import DecisionTreeClassifier # import Decision Tree
from sklearn.ensemble import RandomForestClassifier # import random forest
from sklearn.neural_network import MLPClassifier # import neural_network

## Train and Test on Binary(bag of words) features

In [ ]:
# assign naive bayes function to an object
mnb = MultinomialNB()
#The F1-score combines the precision and recall of a classifier into a single metric by taking their harmonic mean.
#It is primarily used to compare the performance of two classifiers. 
#Suppose that classifier A has a higher recall, and classifier B has higher precision. 
#In this case, the F1-scores for both the classifiers can be used to determine which one produces better results

#Support is the number of actual occurrences of the class in the specified dataset
# predict and evaluate naive bayes
mnb_bowbi_predictions, mnb_bowbi_accuracy,mnb_bowbi_result = train_predict_evaluate_model(classifier=mnb,
                                           train_features=bow_train_features_bi,
                                           train_labels=train_labels,
                                           test_features=bow_test_features_bi,
                                           test_labels=test_labels)

#Support is nothing but the number of test samples available for testing(there are 1739 Total test samples available)
#Test samples that belong to class 0 is 1166

[[189 150]
 [129 238]]
15645
              precision    recall  f1-score   support

           1       0.59      0.56      0.58       339
           2       0.61      0.65      0.63       367

    accuracy                           0.60       706
   macro avg       0.60      0.60      0.60       706
weighted avg       0.60      0.60      0.60       706



In [ ]:
# assign decision tree function to an object
dt = DecisionTreeClassifier()

# predict and evaluate decision tree
dt_bowbi_predictions, dt_bowbi_accuracy,dt_bowbi_result = train_predict_evaluate_model(classifier=dt,
                                                               train_features=bow_train_features_bi,
                                                               train_labels=train_labels,
                                                               test_features=bow_test_features_bi,
                                                               test_labels=test_labels)

[[205 134]
 [182 185]]
14310
              precision    recall  f1-score   support

           1       0.53      0.60      0.56       339
           2       0.58      0.50      0.54       367

    accuracy                           0.55       706
   macro avg       0.55      0.55      0.55       706
weighted avg       0.56      0.55      0.55       706



In [ ]:
# assign random forest function to an object
rf = RandomForestClassifier(criterion="entropy")

# predict and evaluate random forest
rf_bowbi_predictions, rf_bowbi_accuracy,rf_bowbi_result = train_predict_evaluate_model(classifier=rf,
                                           train_features=bow_train_features_bi,
                                           train_labels=train_labels,
                                           test_features=bow_test_features_bi,
                                           test_labels=test_labels)

[[199 140]
 [144 223]]
14720
              precision    recall  f1-score   support

           1       0.58      0.59      0.58       339
           2       0.61      0.61      0.61       367

    accuracy                           0.60       706
   macro avg       0.60      0.60      0.60       706
weighted avg       0.60      0.60      0.60       706



In [ ]:
# assign neural_network function to an object
nn = MLPClassifier()

# predict and evaluate random forest
nn_bowbi_predictions, nn_bowbi_accuracy,nn_bowbi_result = train_predict_evaluate_model(classifier=nn,
                                           train_features=bow_train_features_bi,
                                           train_labels=train_labels,
                                           test_features=bow_test_features_bi,
                                           test_labels=test_labels)

[[183 156]
 [134 233]]
16270
              precision    recall  f1-score   support

           1       0.58      0.54      0.56       339
           2       0.60      0.63      0.62       367

    accuracy                           0.59       706
   macro avg       0.59      0.59      0.59       706
weighted avg       0.59      0.59      0.59       706



## Train and Test on Frequency(bag of words) features

In [ ]:
# assign naive bayes function to an object
mnb = MultinomialNB()
#The F1-score combines the precision and recall of a classifier into a single metric by taking their harmonic mean.
#It is primarily used to compare the performance of two classifiers. 
#Suppose that classifier A has a higher recall, and classifier B has higher precision. 
#In this case, the F1-scores for both the classifiers can be used to determine which one produces better results

#Support is the number of actual occurrences of the class in the specified dataset
# predict and evaluate naive bayes
mnb_bow_predictions, mnb_bow_accuracy,mnb_bow_result = train_predict_evaluate_model(classifier=mnb,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels)

#Support is nothing but the number of test samples available for testing(there are 1739 Total test samples available)
#Test samples that belong to class 0 is 1166

[[177 162]
 [134 233]]
16870
              precision    recall  f1-score   support

           1       0.57      0.52      0.54       339
           2       0.59      0.63      0.61       367

    accuracy                           0.58       706
   macro avg       0.58      0.58      0.58       706
weighted avg       0.58      0.58      0.58       706



In [ ]:
# assign decision tree function to an object
dt = DecisionTreeClassifier()

# predict and evaluate decision tree
dt_bow_predictions, dt_bow_accuracy,dt_bow_result = train_predict_evaluate_model(classifier=dt,
                                                               train_features=bow_train_features,
                                                               train_labels=train_labels,
                                                               test_features=bow_test_features,
                                                               test_labels=test_labels)

[[197 142]
 [158 209]]
14990
              precision    recall  f1-score   support

           1       0.55      0.58      0.57       339
           2       0.60      0.57      0.58       367

    accuracy                           0.58       706
   macro avg       0.58      0.58      0.57       706
weighted avg       0.58      0.58      0.58       706



In [ ]:
# assign random forest function to an object
rf = RandomForestClassifier(criterion="entropy")

# predict and evaluate random forest
rf_bow_predictions, rf_bow_accuracy,rf_bow_result = train_predict_evaluate_model(classifier=rf,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels)

[[207 132]
 [142 225]]
13910
              precision    recall  f1-score   support

           1       0.59      0.61      0.60       339
           2       0.63      0.61      0.62       367

    accuracy                           0.61       706
   macro avg       0.61      0.61      0.61       706
weighted avg       0.61      0.61      0.61       706



In [ ]:
# assign neural_network function to an object
nn = MLPClassifier()

# predict and evaluate random forest
nn_bow_predictions, nn_bow_accuracy, nn_bow_result = train_predict_evaluate_model(classifier=nn,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels)

[[203 136]
 [151 216]]
14355
              precision    recall  f1-score   support

           1       0.57      0.60      0.59       339
           2       0.61      0.59      0.60       367

    accuracy                           0.59       706
   macro avg       0.59      0.59      0.59       706
weighted avg       0.59      0.59      0.59       706



## Train and Test on TFIDF features

In [ ]:
# predict and evaluate naive bayes
mnb_tfidf_predictions, mnb_tfidf_accuracy,mnb_tfidf_result = train_predict_evaluate_model(classifier=mnb,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

[[188 151]
 [143 224]]
15815
              precision    recall  f1-score   support

           1       0.57      0.55      0.56       339
           2       0.60      0.61      0.60       367

    accuracy                           0.58       706
   macro avg       0.58      0.58      0.58       706
weighted avg       0.58      0.58      0.58       706



In [ ]:
# predict and evaluate decision tree
dt_tfidf_predictions, dt_tfidf_accuracy,dt_tfidf_result = train_predict_evaluate_model(classifier=dt,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

[[206 133]
 [192 175]]
14260
              precision    recall  f1-score   support

           1       0.52      0.61      0.56       339
           2       0.57      0.48      0.52       367

    accuracy                           0.54       706
   macro avg       0.54      0.54      0.54       706
weighted avg       0.54      0.54      0.54       706



In [ ]:
# predict and evaluate random forest
rf_tfidf_predictions, rf_tfidf_accuracy,rf_tfidf_result = train_predict_evaluate_model(classifier=rf,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

[[218 121]
 [141 226]]
12805
              precision    recall  f1-score   support

           1       0.61      0.64      0.62       339
           2       0.65      0.62      0.63       367

    accuracy                           0.63       706
   macro avg       0.63      0.63      0.63       706
weighted avg       0.63      0.63      0.63       706



In [ ]:
# predict and evaluate neural_network
nn_tfidf_predictions, nn_tfidf_accuracy,nn_tfidf_result = train_predict_evaluate_model(classifier=nn,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

[[191 148]
 [149 218]]
15545
              precision    recall  f1-score   support

           1       0.56      0.56      0.56       339
           2       0.60      0.59      0.59       367

    accuracy                           0.58       706
   macro avg       0.58      0.58      0.58       706
weighted avg       0.58      0.58      0.58       706



In [ ]:
type(rf_tfidf_predictions)

numpy.ndarray

In [ ]:
rf_tfidf_accuracy

0.6288951841359773

## Visualize Accuracy Matrix

In [ ]:
# create a dictionary that stores all the accuracy information
accuracy_dict = {}
for m in ["mnb","dt","rf","nn"]:
    accuracy_dict[m] = {}
    for f in ["bowbi","bow","tfidf"]:
        exec('accuracy_dict["{}"]["{}"] = {}_{}_accuracy'.format(m, f, m, f))
        
#Accuracy Matrix
pd.DataFrame(accuracy_dict).rename(columns={"mnb":"Naive Bayes", 
                                            "dt":"Decision Tree", 
                                            "rf":"Random Forest",
                                           "nn":"Neural Network"}, 
                                   index={"bowbi":"Binary",
                                          "bow":"Frequency", 
                                          "tfidf":"TFIDF" 
                                          })

,Naive Bayes,Decision Tree,Random Forest,Neural Network
Binary,0.604816,0.552408,0.597734,0.589235
Frequency,0.580737,0.575071,0.611898,0.593484
TFIDF,0.583569,0.539660,0.628895,0.579320


## Visualize Cost Matrix

In [ ]:
# create a dictionary that stores all the accuracy information
cost_dict = {}
for m in ["mnb","dt","rf","nn"]:
    cost_dict[m] = {}
    for f in ["bowbi","bow","tfidf"]:
        exec('cost_dict["{}"]["{}"] = ({}_{}_result)/len(test_labels)'.format(m, f, m, f))
        
#Accuracy Matrix
pd.DataFrame(cost_dict).rename(columns={"mnb":"Naive Bayes", 
                                            "dt":"Decision Tree", 
                                            "rf":"Random Forest",
                                           "nn":"Neural Network"}, 
                                   index={"bowbi":"Binary",
                                          "bow":"Frequency", 
                                          "tfidf":"TFIDF" 
                                          })

,Naive Bayes,Decision Tree,Random Forest,Neural Network
Binary,0.001766,0.001774,0.001520,0.002048
Frequency,0.001732,0.001996,0.001532,0.001769
TFIDF,0.000887,0.002049,0.001694,0.002219


In [ ]:
len(test_labels)

1739

## Set Cross-Valiadation

In [ ]:
from sklearn.model_selection import cross_validate

## Binary, Frequency & TFIDF vector

In [ ]:
bow_vectorizercv_bi=CountVectorizer(min_df=30, max_df=0.90,max_features=5000,ngram_range=(1,1),binary=True)
# print(bow_vectorizer)
#In short, fit is used to train the feature model, once it's trained you can use that model to apply to the text data
bow_train_featurescv_bi = bow_vectorizercv_bi.fit_transform(data["MESSAGE"])
#apply model to test data
bow_test_featurescv_bi = bow_vectorizercv_bi.transform(data["MESSAGE"])

In [ ]:
count_arraycv_bi = bow_train_featurescv_bi.toarray()
dfcv_bi = pd.DataFrame(data=count_arraycv_bi,columns = bow_vectorizercv_bi.get_feature_names())
print(dfcv_bi.shape)
dfcv_bi.head(20)

(5796, 3167)


,aa,ab,abandon,abil,abl,about,abov,absolut,abus,ac,...,zero,zip,zm,zone,zs,zt,zv,zvbnq,zw,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
bow_vectorizercv=CountVectorizer(min_df=30, max_df=0.90,max_features=5000,ngram_range=(1,1))
# print(bow_vectorizer)
#In short, fit is used to train the feature model, once it's trained you can use that model to apply to the text data
bow_train_featurescv = bow_vectorizercv.fit_transform(data["MESSAGE"])
#apply model to test data
bow_test_featurescv = bow_vectorizercv.transform(data["MESSAGE"])

In [ ]:
count_arraycv = bow_train_featurescv.toarray()
dfcv = pd.DataFrame(data=count_arraycv,columns = bow_vectorizercv.get_feature_names())
print(dfcv.shape)
dfcv.head(20)

(5796, 3167)


,aa,ab,abandon,abil,abl,about,abov,absolut,abus,ac,...,zero,zip,zm,zone,zs,zt,zv,zvbnq,zw,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# build tfidf features' vectorizer and get features\
#Min_df stands for minimum document frequency,the less the number, the more specific the term
#‘l2’: Sum of squares of vector elements is 1

tfidf_vectorizercv=TfidfVectorizer(min_df=30, 
                                 max_df=0.90,
                                 max_features=5000,
                                 norm='l2',
                                 smooth_idf=True,
#                                  th_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
tfidf_train_featurescv = tfidf_vectorizercv.fit_transform(data["MESSAGE"])  
tfidf_test_featurescv = tfidf_vectorizercv.transform(data["MESSAGE"])  

In [ ]:
count_array1cv = tfidf_test_featurescv.toarray()
dfcv = pd.DataFrame(data=count_array1cv,columns = tfidf_vectorizercv.get_feature_names())
print(dfcv.shape)
dfcv.head(20)

#Different from count vectorizer. The scores are normalized to values between 0 and 1 
#so the vectors can  be used directly with machine learning algorithms.

(5796, 3167)


,aa,ab,abandon,abil,abl,about,abov,absolut,abus,ac,...,zero,zip,zm,zone,zs,zt,zv,zvbnq,zw,zzzz
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
6,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.029679,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
7,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
8,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9,0.0,0.0,0.0,0.0,0.0,0.007053,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


## Train and Test on Binary(bag of words) features

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
y_pred_nn_bi = cross_val_predict(nn, bow_test_featurescv_bi,data["CATEGORY"] , cv=10) 

In [ ]:
result_nn_bi = metrics.confusion_matrix(data["CATEGORY"],y_pred_nn_bi)
result_nn_bi

array([[3893,    7],
       [  25, 1871]])

In [ ]:
print(metrics.classification_report(data["CATEGORY"],y_pred_nn_bi))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3900
           1       1.00      0.99      0.99      1896

    accuracy                           0.99      5796
   macro avg       0.99      0.99      0.99      5796
weighted avg       0.99      0.99      0.99      5796



In [ ]:
accuracy_nn_bi = metrics.accuracy_score(data["CATEGORY"],y_pred_nn_bi)
cost_nn_bi = result_nn_bi[0][1]*100+result_nn_bi[1][0]*5
cost_nn_bi = cost_nn_bi/len(data["CATEGORY"])

In [ ]:
y_pred_rf_bi = cross_val_predict(rf, bow_test_featurescv_bi,data["CATEGORY"] , cv=10) 
result_rf_bi = metrics.confusion_matrix(data["CATEGORY"],y_pred_rf_bi)
result_rf_bi
print(metrics.classification_report(data["CATEGORY"],y_pred_rf_bi))
accuracy_rf_bi = metrics.accuracy_score(data["CATEGORY"],y_pred_rf_bi)
cost_rf_bi = result_rf_bi[0][1]*100+result_rf_bi[1][0]*5
cost_rf_bi = cost_rf_bi/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3900
           1       0.99      0.96      0.98      1896

    accuracy                           0.98      5796
   macro avg       0.99      0.98      0.98      5796
weighted avg       0.98      0.98      0.98      5796



In [ ]:
y_pred_dt_bi = cross_val_predict(dt, bow_test_featurescv_bi,data["CATEGORY"] , cv=10) 
result_dt_bi = metrics.confusion_matrix(data["CATEGORY"],y_pred_dt_bi)
print(metrics.classification_report(data["CATEGORY"],y_pred_dt_bi))
accuracy_dt_bi = metrics.accuracy_score(data["CATEGORY"],y_pred_dt_bi)
cost_dt_bi = result_dt_bi[0][1]*100+result_dt_bi[1][0]*5
cost_dt_bi = cost_dt_bi/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      3900
           1       0.91      0.94      0.93      1896

    accuracy                           0.95      5796
   macro avg       0.94      0.95      0.94      5796
weighted avg       0.95      0.95      0.95      5796



In [ ]:
y_pred_mnb_bi = cross_val_predict(mnb, bow_test_featurescv_bi,data["CATEGORY"] , cv=10) 
result_mnb_bi = metrics.confusion_matrix(data["CATEGORY"],y_pred_mnb_bi)
print(metrics.classification_report(data["CATEGORY"],y_pred_mnb_bi))
accuracy_mnb_bi = metrics.accuracy_score(data["CATEGORY"],y_pred_mnb_bi)
cost_mnb_bi = result_mnb_bi[0][1]*100+result_mnb_bi[1][0]*5
cost_mnb_bi = cost_mnb_bi/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      3900
           1       0.99      0.95      0.97      1896

    accuracy                           0.98      5796
   macro avg       0.98      0.97      0.98      5796
weighted avg       0.98      0.98      0.98      5796



## Train and Test on Frequency(bag of words) features

In [ ]:
y_pred_nn_fr = cross_val_predict(nn, bow_test_featurescv,data["CATEGORY"] , cv=10) 
result_nn_fr = metrics.confusion_matrix(data["CATEGORY"],y_pred_nn_fr)
print(metrics.classification_report(data["CATEGORY"],y_pred_nn_fr))
accuracy_nn_fr = metrics.accuracy_score(data["CATEGORY"],y_pred_nn_fr)
cost_nn_fr = result_nn_fr[0][1]*100+result_nn_fr[1][0]*5
cost_nn_fr = cost_nn_fr/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3900
           1       1.00      0.98      0.99      1896

    accuracy                           0.99      5796
   macro avg       0.99      0.99      0.99      5796
weighted avg       0.99      0.99      0.99      5796



In [ ]:
y_pred_rf_fr = cross_val_predict(rf, bow_test_featurescv,data["CATEGORY"] , cv=10) 
result_rf_fr = metrics.confusion_matrix(data["CATEGORY"],y_pred_rf_fr)
print(metrics.classification_report(data["CATEGORY"],y_pred_rf_fr))
accuracy_rf_fr = metrics.accuracy_score(data["CATEGORY"],y_pred_rf_fr)
cost_rf_fr = result_rf_fr[0][1]*100+result_rf_fr[1][0]*5
cost_rf_fr = cost_rf_fr/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3900
           1       0.99      0.96      0.97      1896

    accuracy                           0.98      5796
   macro avg       0.99      0.98      0.98      5796
weighted avg       0.98      0.98      0.98      5796



In [ ]:
y_pred_dt_fr = cross_val_predict(dt, bow_test_featurescv,data["CATEGORY"] , cv=10) 
result_dt_fr = metrics.confusion_matrix(data["CATEGORY"],y_pred_dt_fr)
print(metrics.classification_report(data["CATEGORY"],y_pred_dt_fr))
accuracy_dt_fr = metrics.accuracy_score(data["CATEGORY"],y_pred_dt_fr)
cost_dt_fr = result_dt_fr[0][1]*100+result_dt_fr[1][0]*5
cost_dt_fr = cost_dt_fr/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.97      0.96      0.97      3900
           1       0.92      0.94      0.93      1896

    accuracy                           0.95      5796
   macro avg       0.95      0.95      0.95      5796
weighted avg       0.95      0.95      0.95      5796



In [ ]:
y_pred_mnb_fr = cross_val_predict(mnb, bow_test_featurescv,data["CATEGORY"] , cv=10) 
result_mnb_fr = metrics.confusion_matrix(data["CATEGORY"],y_pred_mnb_fr)
print(metrics.classification_report(data["CATEGORY"],y_pred_mnb_fr))
accuracy_mnb_fr = metrics.accuracy_score(data["CATEGORY"],y_pred_mnb_fr)
cost_mnb_fr = result_mnb_fr[0][1]*100+result_mnb_fr[1][0]*5
cost_mnb_fr = cost_mnb_fr/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.90      0.99      0.95      3900
           1       0.99      0.78      0.87      1896

    accuracy                           0.92      5796
   macro avg       0.94      0.89      0.91      5796
weighted avg       0.93      0.92      0.92      5796



## Train and Test on TFIDF features

In [ ]:
y_pred_nn_tifdf = cross_val_predict(nn, tfidf_test_featurescv,data["CATEGORY"] , cv=10) 
result_nn_tifdf = metrics.confusion_matrix(data["CATEGORY"],y_pred_nn_tifdf)
print(metrics.classification_report(data["CATEGORY"],y_pred_nn_tifdf))
accuracy_nn_tifdf = metrics.accuracy_score(data["CATEGORY"],y_pred_nn_tifdf)
cost_nn_tifdf = result_nn_tifdf[0][1]*100+result_nn_tifdf[1][0]*5
cost_nn_tifdf = cost_nn_tifdf/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3900
           1       0.99      0.99      0.99      1896

    accuracy                           0.99      5796
   macro avg       0.99      0.99      0.99      5796
weighted avg       0.99      0.99      0.99      5796



In [ ]:
y_pred_rf_tifdf = cross_val_predict(rf, tfidf_test_featurescv,data["CATEGORY"] , cv=10) 
result_rf_tifdf = metrics.confusion_matrix(data["CATEGORY"],y_pred_rf_tifdf)
print(metrics.classification_report(data["CATEGORY"],y_pred_rf_tifdf))
accuracy_rf_tifdf = metrics.accuracy_score(data["CATEGORY"],y_pred_rf_tifdf)
cost_rf_tifdf = result_rf_tifdf[0][1]*100+result_rf_tifdf[1][0]*5
cost_rf_tifdf = cost_rf_tifdf/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3900
           1       0.99      0.95      0.97      1896

    accuracy                           0.98      5796
   macro avg       0.98      0.97      0.98      5796
weighted avg       0.98      0.98      0.98      5796



In [ ]:
y_pred_dt_tifdf = cross_val_predict(dt, tfidf_test_featurescv,data["CATEGORY"] , cv=10) 
result_dt_tifdf = metrics.confusion_matrix(data["CATEGORY"],y_pred_dt_tifdf)
print(metrics.classification_report(data["CATEGORY"],y_pred_dt_tifdf))
accuracy_dt_tifdf = metrics.accuracy_score(data["CATEGORY"],y_pred_dt_tifdf)
cost_dt_tifdf = result_dt_tifdf[0][1]*100+result_dt_tifdf[1][0]*5
cost_dt_tifdf = cost_dt_tifdf/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      3900
           1       0.92      0.94      0.93      1896

    accuracy                           0.95      5796
   macro avg       0.94      0.95      0.95      5796
weighted avg       0.95      0.95      0.95      5796



In [ ]:
y_pred_mnb_tifdf = cross_val_predict(mnb, tfidf_test_featurescv,data["CATEGORY"] , cv=10) 
result_mnb_tifdf = metrics.confusion_matrix(data["CATEGORY"],y_pred_mnb_tifdf)
print(metrics.classification_report(data["CATEGORY"],y_pred_mnb_tifdf))
accuracy_mnb_tifdf = metrics.accuracy_score(data["CATEGORY"],y_pred_mnb_tifdf)
cost_mnb_tifdf = result_mnb_tifdf[0][1]*100+result_mnb_tifdf[1][0]*5
cost_mnb_tifdf = cost_mnb_tifdf/len(data["CATEGORY"])

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3900
           1       0.99      0.92      0.95      1896

    accuracy                           0.97      5796
   macro avg       0.97      0.96      0.96      5796
weighted avg       0.97      0.97      0.97      5796



## Visualize Accuracy Matrix

In [ ]:
# create a dictionary that stores all the accuracy information
accuracy_dict = {}
for m in ["mnb","dt","rf","nn"]:
    accuracy_dict[m] = {}
    for f in ["bi","fr","tifdf"]:
        exec('accuracy_dict["{}"]["{}"] = accuracy_{}_{}'.format(m, f, m, f))
        
#Accuracy Matrix
pd.DataFrame(accuracy_dict).rename(columns={"mnb":"Naive Bayes", 
                                            "dt":"Decision Tree", 
                                            "rf":"Random Forest",
                                           "nn":"Neural Network"}, 
                                   index={"bi":"Binary",
                                          "fr":"Frequency", 
                                          "tifdf":"TFIDF" 
                                          })

,Naive Bayes,Decision Tree,Random Forest,Neural Network
Binary,0.978261,0.950138,0.984300,0.994479
Frequency,0.923741,0.953934,0.983437,0.993099
TFIDF,0.969289,0.952899,0.981021,0.992581


## Visualize Cost Matrix

In [ ]:
# create a dictionary that stores all the accuracy information
cost_dict = {}
for m in ["mnb","dt","rf","nn"]:
    cost_dict[m] = {}
    for f in ["bi","fr","tifdf"]:
        exec('cost_dict["{}"]["{}"] = cost_{}_{}'.format(m, f, m, f))
        
#Accuracy Matrix
pd.DataFrame(cost_dict).rename(columns={"mnb":"Naive Bayes", 
                                            "dt":"Decision Tree", 
                                            "rf":"Random Forest",
                                           "nn":"Neural Network"}, 
                                   index={"bi":"Binary",
                                          "fr":"Frequency", 
                                          "tifdf":"TFIDF" 
                                          })

,Naive Bayes,Decision Tree,Random Forest,Neural Network
Binary,0.485680,3.134058,0.340752,0.142340
Frequency,0.709110,2.721705,0.377847,0.165631
TFIDF,0.546929,2.726881,0.324362,0.299344


## def a function to do the modeling(trying)

In [ ]:
# def function_cross_validation_model (model,content,label,cv):
#     y_pred = cross_val_predict(model,content,label,cv)
#     result = metrics.confusion_matrix(label,y_pred)
#     print(metrics.confusion_matrix(label,y_pred))
#     cost = result[0][1]*100+result[1][0]*5
#     print(cost)
#     print(metrics.classification_report(label,y_pred))
#     return metrics.accuracy_score(label,y_pred),cost

In [ ]:
# r1,r2 = function_cross_validation_model (model=rf,content = bow_test_featurescv_bi,label =data["CATEGORY"],cv = 10)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


TypeError: Singleton array array(10) cannot be considered a valid collection.